## 1) Текстовое описание набора данных

В качестве набора данных мы будем использовать набор данных 
по прогнозу поступления в Аспирантуру

Датасет содержит следующие колонки:

-GRE Scores 

-TOEFL Scores 

-University Rating 

-Statement of Purpose and Letter of Recommendation Strength 

-Undergraduate GPA 

-Research Experience 

-Chance of Admit 

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.impute import MissingIndicator

: 

In [ ]:
data = pd.read_csv('data/Admission_Predict.csv', sep=";")

: 

In [ ]:
# Проверим наличие пустых значений
# Цикл по колонкам датасета
for col in data.columns:
    # Количество пустых значений - все значения заполнены
    temp_null_count = data[data[col].isnull()].shape[0]
    percentage = 100.0 * temp_null_count / data[col].size
    print('{} - {} ({:.2f}%)'.format(col, temp_null_count, percentage))

: 

# 2) Обработка пропусков в количественном признаке

In [ ]:
sort_data=data['University Rating'].unique()
sort_data.sort()
print(sort_data)

: 

In [ ]:
sort_data_fix=data['University Rating'].unique()
sort_data_fix.sort()
print(sort_data_fix)

: 

In [ ]:
plt.hist(data['University Rating'], bins=40, range=(1,5))

: 

In [ ]:

strategies=['mean', 'median', 'most_frequent']
def test_num_impute_col(dataset, column, strategy_param):
    temp_data = dataset[[column]]
    
    indicator = MissingIndicator()
    mask_missing_values_only = indicator.fit_transform(temp_data)
    
    imp_num = SimpleImputer(strategy=strategy_param)
    data_num_imp = imp_num.fit_transform(temp_data)
    
    filled_data = data_num_imp[mask_missing_values_only]
    return column, strategy_param, filled_data.size, filled_data[0], filled_data[filled_data.size-1]

: 

In [ ]:
print(test_num_impute_col(data, 'University Rating', strategies[0]))
print(test_num_impute_col(data, 'University Rating', strategies[1]))
print(test_num_impute_col(data, 'University Rating', strategies[2]))

: 

Из гистограммы видно, что наиболее предпочтительнее использовать метод 'mean'.
Сравним полученные гистограммы

In [ ]:
fig, ax = plt.subplots(1, 2, sharex='col', sharey='row', figsize=(14,5))
ax[0].hist(data['University Rating'], bins=40, range=(1,5))
ax[1].hist(data['University Rating'].fillna(data['University Rating'].mean()), bins=40, range=(1,5))
ax[0].set_title('with NaN')
ax[1].set_title('without NaN')

: 

In [ ]:

data['University Rating'] = data['University Rating'].fillna(data['University Rating'].mean())

: 

# 3) Обработка пропусков в категориальном признаке

В датасете отсутствует таблица с категориальным признаком, поэтому создадим его на основе числового по оценкам CGPA: 0-8 баллов - Low Grade; 
8-9 - Median Grade; 9-10 - High Grade

In [ ]:
data['Results'] = pd.cut(data['CGPA'], bins=[0, 8, 9, float('Inf')], labels=['Low Grade', 'Median Grade', 'High Grade'])
data['Results'].unique()

: 

In [ ]:
print('Кол-во пропусков: ',data[data['Results'].isnull()].shape[0])
data['Results'].value_counts()

: 

Пропуски можно заполнить значением 'Low Grade'

In [ ]:
data_imp = data['Results'].fillna('Low Grade')

: 

In [ ]:
print('Кол-во пропусков: ',data_imp[data['Results'].fillna('Low Grade').isnull()].shape[0])
data['Results'].fillna('Low Grade').value_counts()

: 

Заполним пропуски

In [ ]:
data['Results'] = data['Results'].fillna('Low Grade')

: 

# 4) График "Скрипичная диаграмма"

In [ ]:
sns.violinplot(x=data['GRE Score'])

: 